In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import simps
from scipy.linalg import solve
from numpy.random import default_rng

rng = default_rng()

# -- Parameters
theta_min, theta_max, N_theta = -10, 10, 200
theta_grid = np.linspace(theta_min, theta_max, N_theta)
delta_theta = theta_grid[1] - theta_grid[0]

# Fermi occupation function (example)
def epsilon(theta): return theta**2 / 2.0  # Free fermions
def n_fermi(theta): return 1.0 / (1.0 + np.exp(epsilon(theta)))
n_theta = n_fermi(theta_grid)

# 1. Monte Carlo sampling of Bethe roots
def monte_carlo_sampling(n_theta, theta_grid, num_samples=1000):
    samples = []
    for _ in range(num_samples):
        occupancy = rng.random(len(n_theta)) < n_theta
        hist, _ = np.histogram(theta_grid[occupancy], bins=theta_grid)
        samples.append(hist)
    samples = np.array(samples)
    mean_rho = np.mean(samples, axis=0)
    var_rho = np.var(samples, axis=0)
    return mean_rho, var_rho

# 2. Langevin GHD (simplified diffusive model)
def ghd_stochastic(theta_grid, n_theta, timesteps=100, dt=0.1):
    rho = n_theta.copy()
    D_theta = rho * (1 - n_theta) * (theta_grid**2)  # Toy diffusion term
    history = [rho.copy()]
    for _ in range(timesteps):
        noise = rng.normal(0, 1, size=rho.shape) * np.sqrt(D_theta * dt / delta_theta)
        rho += np.gradient(D_theta * np.gradient(rho, delta_theta), delta_theta) * dt + noise
        history.append(rho.copy())
    return np.array(history)

# 3. Thermodynamic fluctuation matrix
def dressed_function(f, n_theta, kernel):
    A = np.eye(N_theta) - delta_theta * kernel * n_theta[None, :]
    return solve(A, f)

def compute_variance(f, n_theta, kernel):
    f_dr = dressed_function(f, n_theta, kernel)
    integrand = n_theta * (1 - n_theta) * f_dr**2
    return simps(integrand, theta_grid)

# Kernel (free fermions: zero kernel; interacting case can be added)
kernel = np.zeros((N_theta, N_theta))  # For free fermions: φ = 0

# Run all three methods
mean_rho_mc, var_rho_mc = monte_carlo_sampling(n_theta, theta_grid)
rho_stoch_history = ghd_stochastic(theta_grid, n_theta)
f = theta_grid  # Linear function
var_thermo = compute_variance(f, n_theta, kernel)

# Output for plotting
mean_rho_mc[:5], var_rho_mc[:5], rho_stoch_history[-1][:5], var_thermo


/var/folders/sc/1jlhfymx3sg5vz8q1fx87pvr0000gn/T/ipykernel_97427/839513856.py:50: DeprecationWarning: 'scipy.integrate.simps' is deprecated in favour of 'scipy.integrate.simpson' and will be removed in SciPy 1.14.0
  return simps(integrand, theta_grid)


(array([0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0.]),
 array([ 2.31246660e-09, -1.35751369e-09, -1.68142960e-09,  1.52424894e-10,
        -8.62624515e-09]),
 1.516256042886594)